<a href="https://colab.research.google.com/github/KozinIS/CV/blob/main/01_%D0%92%D1%8B%D1%8F%D0%B2%D0%BB%D0%B5%D0%BD%D0%B8%D0%B5_%D0%BC%D0%BE%D1%88%D0%B5%D0%BD%D0%B5%D1%87%D0%B5%D1%81%D0%BA%D0%B8%D1%85_%D0%B1%D0%B0%D0%BD%D0%BA%D0%BE%D0%B2%D1%81%D0%BA%D0%B8%D1%85_%D1%82%D1%80%D0%B0%D0%BD%D0%B7%D0%B0%D0%BA%D1%86%D0%B8%D0%B9_(AE).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Выявление мошенеческих банковских транзакций (AE)**

# 1 Постановка задачи

*Добейтесь точности не менее 90 % в решении задачи поиска мошеннических операций*

*Попробуйте сделать автокодировщик со свёрточными слоями. Сделайте гибкую настройку порога (bias), по которому мы определяем, мошенническая транзакция или нет.*

# 2 Подгрузка библиотек и подключение диска

In [ ]:
from tensorflow.keras.models import Model, Sequential      # Абстрактный класс базовой модели сети от кераса и последовательная модель

# Из кераса загружаем необходимые слои для нейросети:
from tensorflow.keras.layers import Dense, Dropout, Flatten, Reshape, Input, Conv2DTranspose, Conv1DTranspose, concatenate, Activation, MaxPooling2D, MaxPooling1D, Conv2D, Conv1D, BatchNormalization
from tensorflow.keras import backend as K                  # Базовые керасовские функции
from tensorflow.keras.optimizers import Adam               # Оптимизатор Адам
from tensorflow.keras import utils                         # Утилиты кераса
from tensorflow.keras.datasets import mnist, fashion_mnist # Готовые базы mnist

import matplotlib.pyplot as plt                            # Из библиотеки для визуализации данных возьмём интерфейс для построения графиков простых функций
from tensorflow.keras.preprocessing import image           # Модуль для отрисовки изображения
import numpy as np                                         # Библиотека для работы с массивами данных
import pandas as pd                                        # Библиотека для анализа и обработки данных
from PIL import Image                                      # Модуль для отрисовки изображения
from sklearn.model_selection import train_test_split       # Модуль для разбивки выборки на тренировочную/тестовую
from sklearn.preprocessing import StandardScaler           # Модуль для стандартизации данных

import os
from google.colab import drive
import time

In [ ]:
drive.mount('/content/drive/')

Mounted at /content/drive/


# 3 Подгрузка базы и создание обучеющей выборки

In [ ]:
df = pd.read_csv("/content/drive/My Drive/HW_22/creditcard.csv")

In [ ]:
df.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V15,V16,V17,V18,V19,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,0.090794,-0.551600,-0.617801,-0.991390,-0.311169,1.468177,-0.470401,0.207971,0.025791,0.403993,0.251412,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,-0.166974,1.612727,1.065235,0.489095,-0.143772,0.635558,0.463917,-0.114805,-0.183361,-0.145783,-0.069083,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,0.207643,0.624501,0.066084,0.717293,-0.165946,2.345865,-2.890083,1.109969,-0.121359,-2.261857,0.524980,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,-0.054952,-0.226487,0.178228,0.507757,-0.287924,-0.631418,-1.059647,-0.684093,1.965775,-1.232622,-0.208038,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,0.753074,-0.822843,0.538196,1.345852,-1.119670,0.175121,-0.451449,-0.237033,-0.038195,0.803487,0.408542,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0


In [ ]:
# Удаляем столбец со временем:
data = df.drop(['Time'], axis=1)

# Нормализуем столбец с суммой транзакции:
data['Amount'] = StandardScaler().fit_transform(data['Amount'].values.reshape(-1, 1))

frauds = data[data.Class == 1]  # Записываем мошеннические операции
normal = data[data.Class == 0]  # Записываем нормальные операции

In [ ]:
frauds.shape

(492, 30)

In [ ]:
normal.shape

(284315, 30)

In [ ]:
RANDOM_SEED = 42 # Задаем случайное значение

# Берём нормальные операции и разбиваем на train_test(только 'X'):
X_train, X_test = train_test_split(normal, test_size=0.2, random_state=RANDOM_SEED)

In [ ]:
X_train = X_train.drop(['Class'], axis=1) # Удаляем класс в обучающем наборе данных
X_test = pd.concat([X_test, frauds])      # Добавляем все мошеннические транзакции в тестовый набор данных
y_test = X_test['Class']                  # Делаем метки для тестового набора данных
X_test = X_test.drop(['Class'], axis=1)   # Удаляем класс у тестового набора данных

# Преобразуем данные в массивы numpy:
X_train = X_train.values
X_test = X_test.values

In [ ]:
print(X_train.shape)

(227452, 29)


In [ ]:
X_train[0].shape

(29,)

In [ ]:
X_train.shape[1]

29

In [ ]:
type(X_test)

numpy.ndarray

# 4 Создание и обучение базового автокодировщика

Функция создания автокодировщика

In [ ]:
def creditcardAutoencoder():             # Объявляем автокодировщик для определения мошейнич. транзакций
  dataSize = X_train.shape[1]            # Берём размеры X_train по второй оси (29)
  dataInput = Input(shape=(dataSize, ))  # Задаем эти размеры как входные в сеть

  x = Dense(10, activation='relu')(dataInput)       # Пропускаем через полносвязный слой размером 10
  x = Dense(dataSize, activation='linear')(x)       # и через полносвязный слой размером 29

  autoencoder = Model(inputs=dataInput, outputs=x)  # Сборка модели

  autoencoder.compile(optimizer='Adam', loss='mse') # Компилируем модель с выбором оптимайзера и среднеквадратичной ошибки

  return autoencoder

Создание и обучение нейросети

In [ ]:
model = creditcardAutoencoder()

model.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 29)]              0         
_________________________________________________________________
dense_2 (Dense)              (None, 10)                300       
_________________________________________________________________
dense_3 (Dense)              (None, 29)                319       
Total params: 619
Trainable params: 619
Non-trainable params: 0
_________________________________________________________________


In [ ]:
history = model.fit(X_train, X_train,
                    epochs=10,
                    batch_size=33) # Загружаем данные в модель(X_train'ы) и обучаем

Epoch 1/10
6893/6893 [==============================] - 12s 2ms/step - loss: 0.3570
Epoch 2/10
6893/6893 [==============================] - 12s 2ms/step - loss: 0.3555
Epoch 3/10
6893/6893 [==============================] - 12s 2ms/step - loss: 0.3550
Epoch 4/10
6893/6893 [==============================] - 12s 2ms/step - loss: 0.3546
Epoch 5/10
6893/6893 [==============================] - 12s 2ms/step - loss: 0.3545
Epoch 6/10
6893/6893 [==============================] - 12s 2ms/step - loss: 0.3544
Epoch 7/10
6893/6893 [==============================] - 12s 2ms/step - loss: 0.3542
Epoch 8/10
6893/6893 [==============================] - 12s 2ms/step - loss: 0.3543
Epoch 9/10
6893/6893 [==============================] - 12s 2ms/step - loss: 0.3543
Epoch 10/10
6893/6893 [==============================] - 12s 2ms/step - loss: 0.3542


Результаты

In [ ]:
predictions = model.predict(X_test)                      # делаем предсказание по X_test
mse = np.mean(np.power(X_test - predictions, 2), axis=1) # определяем среднеквадратичную ошибку по X_test и предсказанием по X_test
mse_normal = mse[y_test.values == 0]                     # среднеквадратичная ошибка на нормальных операциях
mse_frauds = mse[y_test.values == 1]                     # среднеквадратичная ошибка на мошеннических операциях
print("Минимальная ошибка нормальных трансакций:", round(min(mse_normal),4))      # найдем минимальную среднеквадратичную ошибку на нормальных операциях
print("Максимальная ошибка нормальных трансакций:", round(max(mse_normal),4))     # найдем максимальную среднеквадратичную ошибку на нормальных операциях
print("Средняя ошибка нормальных трансакций:", round(sum(mse_normal) / len(mse_normal),4)) # среднюю ошибку
print("=================================")
print("Минимальная ошибка мошеннических трансакций:", round(min(mse_frauds),4))   # найдем минимальную среднеквадратичную ошибку на мошеннических операциях
print("Максимальная ошибка мошеннических трансакций:", round(max(mse_frauds),4))  # найдем максимальную среднеквадратичную ошибку на мошеннических операциях
print("Средняя ошибка мошеннических трансакций:", round(sum(mse_frauds) / len(mse_frauds),4))

Минимальная ошибка нормальных трансакций: 0.0403
Максимальная ошибка нормальных трансакций: 337.8032
Средняя ошибка нормальных трансакций: 0.355
Минимальная ошибка мошеннических трансакций: 0.1553
Максимальная ошибка мошеннических трансакций: 97.2615
Средняя ошибка мошеннических трансакций: 18.7157


In [ ]:
def getAccByBias(bias): # Функция будет принимать какое-то пороговое значение:
  isNormal = mse_normal < bias # - если ошибка меньше порога - то транзакция нормальная;
  isFrauds = mse_frauds > bias # - если ошибка больше порога - то транзакция мошенническая.

  accNormal = sum(isNormal) / len(isNormal) # Подсчёт процента нормальных операций
  accFaruds = sum(isFrauds) / len(isFrauds) # Подсчёт процента мошеннических операций

  print("Распознано нормальных трансакций: ", round(100*accNormal), "%", sep="")
  print("Распознано мошеннических трансакций: ", round(100*accFaruds), "%", sep="")
  print("Средняя точность распознавания: ", round(50*(accNormal + accFaruds)), "%", sep="")

In [ ]:
getAccByBias(0.8) # Вывод данных по распознаванию с конкретным пороговым значением

Распознано нормальных трансакций: 96.0%
Распознано мошеннических трансакций: 88.0%
Средняя точность распознавания: 92.0%


# 5 Эксперимент № 1 (с разжимаем и сжимаем)

## 5.1 Создание нейросети

In [ ]:
dataInput = Input(shape=(29,1))  # (29,)

x = Conv1D(32, (3), padding='same', activation='relu')(dataInput)
x = BatchNormalization()(x)
x = Conv1D(32, (3), padding='same', activation='relu')(x)
x = BatchNormalization()(x)

x = Conv1DTranspose(64, (2), strides=(2), padding='same', activation='relu')(x)

x = Conv1D(32, (3), padding='same', activation='relu')(x)
x = BatchNormalization()(x)
x = Conv1D(32, (3), padding='same', activation='relu')(x)
x = BatchNormalization()(x)

x = MaxPooling1D()(x)
x = BatchNormalization()(x)

x = Conv1D(64, (3), padding='same', activation='relu')(x)
x = BatchNormalization()(x)
x = Conv1D(64, (3), padding='same', activation='relu')(x)
x = BatchNormalization()(x)

x = Conv1D(1, (3), activation='sigmoid', padding='same')(x)

autoencoder = Model(inputs=dataInput, outputs=x)
autoencoder.compile(optimizer='Adam', loss='mse')

autoencoder.summary()

Model: "functional_65"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_37 (InputLayer)        [(None, 29, 1)]           0         
_________________________________________________________________
conv1d_71 (Conv1D)           (None, 29, 32)            128       
_________________________________________________________________
batch_normalization_12 (Batc (None, 29, 32)            128       
_________________________________________________________________
conv1d_72 (Conv1D)           (None, 29, 32)            3104      
_________________________________________________________________
batch_normalization_13 (Batc (None, 29, 32)            128       
_________________________________________________________________
conv1d_transpose (Conv1DTran (None, 58, 64)            4160      
_________________________________________________________________
conv1d_73 (Conv1D)           (None, 58, 32)          

## 5.2 Обучение нейросети

In [ ]:
history = autoencoder.fit(X_train, X_train,
                    epochs=5,
                    batch_size=32)

Epoch 1/5
7108/7108 [==============================] - 54s 8ms/step - loss: 0.7087
Epoch 2/5
7108/7108 [==============================] - 54s 8ms/step - loss: 0.7072
Epoch 3/5
7108/7108 [==============================] - 55s 8ms/step - loss: 0.7070
Epoch 4/5
7108/7108 [==============================] - 54s 8ms/step - loss: 0.7070
Epoch 5/5
7108/7108 [==============================] - 54s 8ms/step - loss: 0.7070


## 5.3 Результаты работы нейорсети

In [ ]:
predictions = autoencoder.predict(X_test)  # Делаем предсказание по X_test

predictions_2 = predictions.reshape(57355, 29)

mse = np.mean(np.power(X_test - predictions_2, 2), axis=1)  # Определяем среднеквадратичную ошибку по X_test и предсказанием по X_test
mse_normal = mse[y_test.values == 0]                        # Среднеквадратичная ошибка на нормальных операциях
mse_frauds = mse[y_test.values == 1]                        # Среднеквадратичная ошибка на мошеннических операциях

print("Минимальная ошибка нормальных транзакций:", round(min(mse_normal),4))  # Поиск минимальной среднеквадратичной ошибки на нормальных операциях
print("Максимальная ошибка нормальных транзакций:", round(max(mse_normal),4)) # Поиск максимальной среднеквадратичной ошибки на нормальных операциях
print("Средняя ошибка нормальных транзакций:", round(sum(mse_normal) / len(mse_normal),4))
print("========================================:")
print("Минимальная ошибка мошеннических транзакций:", round(min(mse_frauds),4))  # Поиск минимальной среднеквадратичной ошибки на мошеннических операциях
print("Максимальная ошибка мошеннических транзакций:", round(max(mse_frauds),4)) # Поиск максимальной среднеквадратичной ошибки на мошеннических операциях
print("Средняя ошибка мошеннических транзакций:", round(sum(mse_frauds) / len(mse_frauds),4))

Минимальная ошибка нормальных транзакций: 0.0263
Максимальная ошибка нормальных транзакций: 1853.1406
Средняя ошибка нормальных транзакций: 0.7189
========================================:
Минимальная ошибка мошеннических транзакций: 0.0855
Максимальная ошибка мошеннических транзакций: 275.1559
Средняя ошибка мошеннических транзакций: 26.0601


In [ ]:
def getAccByBias(bias): # Функция будет принимать какое-то пороговое значение:
  isNormal = mse_normal < bias # - если ошибка меньше порога - то транзакция нормальная;
  isFrauds = mse_frauds > bias # - если ошибка больше порога - то транзакция мошенническая.

  accNormal = sum(isNormal) / len(isNormal) # Подсчёт процента нормальных операций
  accFaruds = sum(isFrauds) / len(isFrauds) # Подсчёт процента мошеннических операций

  print("Распознано нормальных трансакций: ", round(100*accNormal), "%", sep="")
  print("Распознано мошеннических трансакций: ", round(100*accFaruds), "%", sep="")
  print("Средняя точность распознавания: ", round(50*(accNormal + accFaruds)), "%", sep="")

In [ ]:
getAccByBias(1) # Вывод данных по распознаванию с конкретным пороговым значением

Распознано нормальных трансакций: 91.0%
Распознано мошеннических трансакций: 90.0%
Средняя точность распознавания: 90.0%


# 6 Эксперимент № 2 (без разжатия/сжатия)

## 6.1 Создание нейросети

In [ ]:
dataInput = Input(shape=(29,1))  # было (29,)

x = Dense(10, activation='relu')(dataInput)

x = Conv1D(64, (2), padding='same', activation='relu')(x)
x = Conv1D(64, (2), padding='same', activation='relu')(x)

x = Conv1D(1, (2), activation='sigmoid', padding='same')(x)

autoencoder = Model(inputs=dataInput, outputs=x)
autoencoder.compile(optimizer='Adam', loss='mse')

autoencoder.summary()

Model: "functional_63"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_36 (InputLayer)        [(None, 29, 1)]           0         
_________________________________________________________________
dense_51 (Dense)             (None, 29, 10)            20        
_________________________________________________________________
conv1d_68 (Conv1D)           (None, 29, 64)            1344      
_________________________________________________________________
conv1d_69 (Conv1D)           (None, 29, 64)            8256      
_________________________________________________________________
conv1d_70 (Conv1D)           (None, 29, 1)             129       
Total params: 9,749
Trainable params: 9,749
Non-trainable params: 0
_________________________________________________________________


## 6.2 Обучение нейросети

In [ ]:
history = autoencoder.fit(X_train, X_train,
                    epochs=3,
                    batch_size=32)

Epoch 1/3
7108/7108 [==============================] - 20s 3ms/step - loss: 0.7069
Epoch 2/3
7108/7108 [==============================] - 20s 3ms/step - loss: 0.7069
Epoch 3/3
7108/7108 [==============================] - 19s 3ms/step - loss: 0.7069


## 6.3 Результаты работы нейорсети

In [ ]:
predictions = autoencoder.predict(X_test)  # Делаем предсказание по X_test

predictions_2 = predictions.reshape(57355, 29)

mse = np.mean(np.power(X_test - predictions_2, 2), axis=1)  # Определяем среднеквадратичную ошибку по X_test и предсказанием по X_test
mse_normal = mse[y_test.values == 0]                        # Среднеквадратичная ошибка на нормальных операциях
mse_frauds = mse[y_test.values == 1]                        # Среднеквадратичная ошибка на мошеннических операциях

print("Минимальная ошибка нормальных транзакций:", round(min(mse_normal),4))     # Поиск минимальной среднеквадратичной ошибки на нормальных операциях
print("Максимальная ошибка нормальных транзакций:", round(max(mse_normal),4))    # Поиск максимальной среднеквадратичной ошибки на нормальных операциях
print("Средняя ошибка нормальных транзакций:", round(sum(mse_normal) / len(mse_normal),4))
print("========================================:")
print("Минимальная ошибка мошеннических транзакций:", round(min(mse_frauds),4))  # Поиск минимальной среднеквадратичной ошибки на мошеннических операциях
print("Максимальная ошибка мошеннических транзакций:", round(max(mse_frauds),4)) # Поиск максимальной среднеквадратичной ошибки на мошеннических операциях
print("Средняя ошибка мошеннических транзакций:", round(sum(mse_frauds) / len(mse_frauds),4))

Минимальная ошибка нормальных транзакций: 0.0263
Максимальная ошибка нормальных транзакций: 1853.1406
Средняя ошибка нормальных транзакций: 0.7189
========================================:
Минимальная ошибка мошеннических транзакций: 0.0855
Максимальная ошибка мошеннических транзакций: 275.1559
Средняя ошибка мошеннических транзакций: 26.0599


In [ ]:
def getAccByBias(bias): # Функция будет принимать какое-то пороговое значение:
  isNormal = mse_normal < bias # - если ошибка меньше порога - то транзакция нормальная;
  isFrauds = mse_frauds > bias # - если ошибка больше порога - то транзакция мошенническая.

  accNormal = sum(isNormal) / len(isNormal) # Подсчёт процента нормальных операций
  accFaruds = sum(isFrauds) / len(isFrauds) # Подсчёт процента мошеннических операций

  print("Распознано нормальных трансакций: ", round(100*accNormal), "%", sep="")
  print("Распознано мошеннических трансакций: ", round(100*accFaruds), "%", sep="")
  print("Средняя точность распознавания: ", round(50*(accNormal + accFaruds)), "%", sep="")

In [ ]:
getAccByBias(1) # Вывод данных по распознаванию с конкретным пороговым значением

Распознано нормальных трансакций: 91.0%
Распознано мошеннических трансакций: 90.0%
Средняя точность распознавания: 90.0%


# 7 Выводы

Создан и обучен автокодировщик со свёрточными слоями, предназначенный для выявления мошенеческих банковсеких транзакций. Средняя точность работы автокодировщика составила 90%.

Эксперементально показано - значительное влияние на точность работы оказывает внесение изменений в размерность входных данных (для Conv1D необходима размерность (29,1)), вследствие которых на выходе нейросети остаётся только один нейрон. При такой размерности на точность работы нейросети не оказывает значительного влияния состав скрытых слоёв и их параметры. В т.ч. была проверена гипотеза о включении в состав нейронной сети скрытых слоёв, "расширяющих" и "сужающих" размер выборок (Conv1DTranspose и MaxPooling1D).